In [1]:
import sys
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\change_of_variable')
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\hmc_pytorch')
from change_of_variable_pytorch import * 
from hmc_base_pytorch import *
from hmc_unconstrained_pytorch import *
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal
import matplotlib.pyplot as plt

import torch.utils.data
from torch import optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import time
import torchvision


cuda = True
batch_size = 64
epochs = 10
seed = 1
z_dim = 20

# Data preparation
torch.manual_seed(seed)
device = torch.device("cuda" if cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_data = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size, shuffle=True, **kwargs)

def binarization(data):
    s = np.random.uniform(size = data.shape)
    out = np.array(s<data).astype(float)
    return out

result = []
for batch_idx, (data, _) in enumerate(train_loader):
    data = data.view(-1, 784).numpy()
    bi_data = binarization(data)
    d = torch.from_numpy(bi_data)
    result.append(d)
    
result_test = []
for batch_idx, (data, _) in enumerate(test_loader):
    data = data.view(-1, 784).numpy()
    bi_data = binarization(data)
    d = torch.from_numpy(bi_data)
    result_test.append(d)

############################################################
def reparameterize(mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    #return eps.mul(std).add_(mu)
    return eps*std+mu

def log_prior(z):
    dim = z.shape[1]
    mean = torch.zeros(dim).cuda()
    cov = torch.eye(dim).cuda()
    m = MultivariateNormal(mean, cov)
    m.requires_grad=True
    return m.log_prob(z)

def multivariate_normal_diagonal_logpdf(mean, cov_diag, x):
    n = x.shape[0] # number of samples
    k = x.shape[1] # dimension
    t1 = -0.5*(x - mean)*(1/cov_diag)*(x-mean)
    t1 = torch.sum(t1, dim=1)
    t2 = torch.ones(n).cuda()*0.5*k*torch.log(torch.tensor([2*math.pi]).cuda()) + 0.5*torch.sum(torch.log(cov_diag),dim=1)
    return t1 - t2
############################################################
class decoder(nn.Module):
    def __init__(self):
        super(decoder, self).__init__()
        self.fc1 = nn.Linear(z_dim, 400)
        self.fc2 = nn.Linear(400, 784)
    # single hidden layer
    def forward(self, x):
        #x = x.view(-1, 784)
        h1 = F.relu(self.fc1(x))
        return F.sigmoid(self.fc2(h1))
    
class q_z0(nn.Module):
    def __init__(self):
        super(q_z0, self).__init__()
        self.fc1 = nn.Linear(784, 300)
        self.fc2 = nn.Linear(300, 300)
        self.fc31 = nn.Linear(300, z_dim)
        self.fc32 = nn.Linear(300, z_dim)
    def forward(self, x):
        x = x.view(-1, 784)
        h1 = F.softplus(self.fc1(x))
        h2 = F.softplus(self.fc2(h1))
        logvar = self.fc31(h2)
        mu = self.fc32(h2)
        return mu, logvar
    
class r_v(nn.Module):
    def __init__(self):
        super(r_v, self).__init__()
        self.fc1 = nn.Linear(z_dim + 784, 300)
        self.fc21 = nn.Linear(300, z_dim)
        self.fc22 = nn.Linear(300, z_dim)
    def forward(self, x):
        x = x.view(-1, 784 + z_dim)
        h1 = F.softplus(self.fc1(x))
        logvar = self.fc21(h1)
        mu = self.fc22(h1)
        return mu, logvar

class q_v(nn.Module):
    def __init__(self):
        super(q_v, self).__init__()
        self.fc1 = nn.Linear(784, 300)
        # no need to output mu because the mean of momentum is default 0
        self.fc21 = nn.Linear(300, z_dim)
    def forward(self, x):
        x = x.view(-1, 784)
        h1 = F.softplus(self.fc1(x))
        logvar = self.fc21(h1)
        return logvar
############################################################
decoder = decoder().to(device)
q_z0 = q_z0().to(device)
r_v = r_v().to(device)
q_v = q_v().to(device)

In [7]:
def ELBO(data, decoder, q_z0, r_v, q_v, log_step, T):#q_z0_mean, q_z0_logvar):
    
    #file1 = open("q_z0_mean"+str(epoch)+".txt","w")
    #file1 = open("q_z0_var"+str(epoch)+".txt","w")
    #file1 = open("z0"+str(epoch)+".txt","w")
    batch_size = data.view(-1, 784).shape[0]
    data = data.to(device)
    
    q_z0_mean, q_z0_logvar = q_z0(data)
    # sample z0
    z0 = reparameterize(q_z0_mean, q_z0_logvar)
    
    # compute q(z0|x)
    var_z0 = torch.exp(q_z0_logvar)
    log_q_z0 = multivariate_normal_diagonal_logpdf(q_z0_mean, var_z0, z0)
    
    #print("q z0 mean: "+str(q_z0_mean))
    #print("q z0 var: "+str(var_z0))
    #print("z0: "+str(z0))
    #print("q(z0|x): "+str(log_q_z0))
    #print("np q(z0|x): "+str(np_log_q_z0))
    
    # compute prior
    log_prior_z0 = log_prior(z0)
    
    
    # compute joint
    decoder_output = decoder(z0)
    log_likelihood = 0. - F.binary_cross_entropy(decoder_output, data.view(-1, 784).float(), size_average=False, reduce=False)
    log_likelihood = torch.sum(log_likelihood, dim = 1)
    log_joint = log_prior_z0 + log_likelihood
    
    
    # initial L
    #L = log_joint - log_q_z0
    L = -log_q_z0
    #print("initial L: "+str(L))
    #print("log likelihood 0: "+str(torch.mean(log_likelihood)))
    
    for i in range(T):
        # sample v1
        
        logvar_v1 = q_v(data)
        
        #logvar_v1 = torch.zeros([batch_size, z_dim], requires_grad = True).cuda()
        var_v1 = torch.exp(logvar_v1)
        mu_v1 = torch.zeros([logvar_v1.shape[0], logvar_v1.shape[1]],requires_grad = True).cuda()
        v1 = reparameterize(mu_v1, logvar_v1)
        #print("v1: "+str(v1.shape))
        
        # compute q(v1|x) 
        log_q_v1 = multivariate_normal_diagonal_logpdf(mu_v1, var_v1 ,v1)
        #print("log_q_v1: "+str(log_q_v1.shape))
        #print("var_v1: "+str(var_v1))
        
        mass_diag = var_v1
        #print("mass_diag: "+str(mass_diag))
        
        def energy_function(z, cache):
            z = z.view(batch_size, z_dim)
            z_cuda = z.cuda()
            all_log_prior = log_prior(z_cuda)
            #print("all prior: "+str(all_log_prior.shape))
            all_log_prior = torch.sum(all_log_prior)
            #print(all_log_prior.shape)
            decoder_output = decoder(z_cuda)

            all_log_likelihood = 0. - F.binary_cross_entropy(decoder_output, data.view(-1, 784).float(), size_average=False)
            #print("one_log_likelihood: "+str(one_log_likelihood.shape))
            all_log_joint = all_log_prior + all_log_likelihood
            return 0 - all_log_joint
        
        
        init = z0.view(batch_size*z_dim)
        mass_diag = mass_diag.view(mass_diag.shape[0]*mass_diag.shape[1])
        mass_matrix = torch.diag(mass_diag)
        mom = v1.view(v1.shape[0]*v1.shape[1])
        step = torch.exp(log_step)
        #print("init: "+str(init.shape))
        #print("mass: "+str(mass_matrix.shape))
        
        # clip step size
        low = 0.0005
        high = 0.5
        torch.clamp(step, min=low, max=high)
        step = step.repeat(1, batch_size)
        step = step.view(step.shape[1])
        step_cuda = step.cuda()
        print("log step: "+str(log_step))
        print("step: "+str(step_cuda))
        
        sampler = IsotropicHmcSampler(energy_function, energy_grad=None, prng=None, mom_resample_coeff=0., dtype=np.float64)
        #pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.0001, 5, 2, mass_matrix, mom = mom)
        pos_samples, mom_samples, ratio = sampler.get_samples(init, step_cuda, 5, 2, mass_matrix, mom = mom)
        
        zt = pos_samples[1].cuda()
        vt = mom_samples[1].cuda()
        
        zt = zt.view(batch_size, z_dim)
        vt = vt.view(batch_size, z_dim)

        # get joint probaility p(x, zt)
        log_prior_zt = log_prior(zt)
        decoder_output_t = decoder(zt)
        #print("log prior: " + str(log_prior_zt.shape))
        #print("decoder: " + str(decoder_output_t.shape))

        log_likelihood_t = 0. - F.binary_cross_entropy(decoder_output_t, data.view(-1, 784).float(), size_average=False, reduce=False)
        log_likelihood_t = torch.sum(log_likelihood_t, dim = 1)
        #print("log ll: " + str(log_likelihood_t.shape))
        
        log_joint_t = log_prior_zt + log_likelihood_t
        #print("log joint t: " + str(log_joint_t.shape))
        
        # get r(vt|x,zt)
        d = data.view(-1, 784)
        #print(d.shape)
        new_data = torch.cat((d.float(), zt), 1) # append data with zt
        #print("new data: "+str(one_new_data.shape))

        mu_vt, logvar_vt = r_v(new_data)
        var_vt = torch.exp(logvar_vt)
        #print("var_vt: "+str(var_vt.shape))

        log_r_vt = multivariate_normal_diagonal_logpdf(mu_vt, var_vt, vt)
        """
        print("log prior t: "+str(torch.mean(log_prior_zt)))
        print("log likelihood t: "+str(torch.mean(log_likelihood_t)))
        print("joint t: "+str(torch.mean(log_joint_t)))
        print("reverse: "+str(torch.mean(log_r_vt)))
        #print(log_joint[j])
        print("forward: "+str(torch.mean(log_q_v1)))
        print("q(z0|x): "+str(torch.mean(log_q_z0)))
        print("========================================`")
        """
        #file1 = open("q_z0_mean"+str(epoch)+".txt","w")
        #file2 = open("q_z0_var"+str(epoch)+".txt","w")
        #file3 = open("z0"+str(epoch)+".txt","w")
        
        # get L for each sample
        #one_log_alpha = one_log_joint_t + one_log_r_vt - log_joint[j] - log_q_v1[j]
        log_alpha = log_joint_t + log_r_vt - log_q_v1
        #one_log_alpha = log_joint[j] #+ one_log_r_vt - log_q_v1[j]

        L = L + log_alpha
        #print("L: "+str(L.shape))
    
    
    return torch.mean(L)

In [11]:
batch_size = 64
z_dim = 20

params1 = list(decoder.parameters())+list(q_z0.parameters())+list(r_v.parameters())+list(q_v.parameters())
optimizer1 = optim.Adam(params1, lr=0.0035)#, weight_decay=1e-4)
#optimizer1 = optim.Adam([q_z0_mean], lr=0.005)
#log_step_raw = torch.ones(z_dim, requires_grad=True)
#log_step = -3.0*log_step_raw
log_step=torch.tensor([-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.], requires_grad=True)
optimizer2 = optim.Adam([log_step], lr=0.003)


for epoch in range(20):
    print("Epoch: "+str(epoch+1))
    file = open("result11_"+str(epoch)+".txt","w")
    file_test = open("result11_test_"+str(epoch)+".txt","w")
    for i in range(len(result)):
        print("++++++++++"+" Epoch: "+str(epoch+1)+" batch: " + str(i) + " ++++++++++")
        data = result[i].float()
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss = 0. - ELBO(data, decoder, q_z0, r_v, q_v, log_step, 1)#_mean_cuda, q_z0_logvar_cuda)
        print("ELBO: "+str(0-loss.item()))
        loss.backward()
        #nn.utils.clip_grad_norm_(q_z0.parameters(), 1)
        print(log_step_raw.grad)
        #print(q_z0.fc1.weight.grad)
        #print(q_z0.fc31.weight.grad)
        #print(q_z0.fc32.weight.grad)
        optimizer1.step()
        optimizer2.step()
        file.write(str(0.-loss.item())+"\n")
    file.close()
    for i in range(len(result_test)):
        print("++++++++++ test batch: " + str(i) + " ++++++++++")
        data = result_test[i].float()
        loss = 0. - ELBO(data, decoder, q_z0, r_v, q_v, log_step, 1)
        print("ELBO: "+str(0-loss.item()))
        file_test.write(str(0.-loss.item())+"\n")
        
    file_test.close()
    
    sample = torch.randn(64, 20).to(device)
    sample = decoder(sample).cpu()
    save_image(sample.view(64, 1, 28, 28), 'sample11_' + str(epoch) + '.png')

Epoch: 1
++++++++++ Epoch: 1 batch: 0 ++++++++++
log step: tensor([-3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3.,
        -3., -3., -3., -3., -3., -3., -3., -3.])
step: tensor(1.00000e-02 *
       [ 4.9787,  4.9787,  4.9787,  ...,  4.9787,  4.9787,  4.9787], device='cuda:0')
ELBO: -130.01388549804688
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 1 ++++++++++
log step: tensor([-3.0030, -3.0030, -3.0030, -3.0030, -2.9970, -2.9970, -3.0030,
        -3.0030, -2.9970, -3.0030, -3.0030, -3.0030, -3.0030, -3.0030,
        -3.0030, -3.0030, -2.9970, -2.9970, -3.0030, -3.0030])
step: tensor(1.00000e-02 *
       [ 4.9638,  4.9638,  4.9638,  ...,  4.9937,  4.9638,  4.9638], device='cuda:0')
ELBO: -140.77127075195312
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 2 ++++++++++
log step: tensor(

ELBO: -137.77484130859375
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 17 ++++++++++
log step: tensor([-3.0021, -3.0407, -3.0382, -3.0210, -3.0341, -2.9948, -3.0454,
        -3.0328, -2.9913, -3.0364, -3.0370, -3.0209, -2.9965, -3.0432,
        -3.0396, -3.0464, -3.0334, -2.9857, -3.0439, -3.0391])
step: tensor(1.00000e-02 *
       [ 4.9684,  4.7801,  4.7923,  ...,  5.0505,  4.7647,  4.7878], device='cuda:0')
ELBO: -137.3838348388672
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 18 ++++++++++
log step: tensor([-3.0028, -3.0427, -3.0402, -3.0234, -3.0368, -2.9935, -3.0480,
        -3.0353, -2.9930, -3.0386, -3.0382, -3.0231, -2.9966, -3.0460,
        -3.0422, -3.0493, -3.0359, -2.9855, -3.0463, -3.0408])
step: tensor(1.00000e-02 *
       [ 4.9648,  4.7707,  4.7826,  ...,  5.0515,  4.7533,  4.7

ELBO: -139.23435974121094
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 34 ++++++++++
log step: tensor([-3.0060, -3.0785, -3.0795, -3.0589, -3.0766, -2.9772, -3.0828,
        -3.0724, -2.9940, -3.0786, -3.0651, -3.0594, -2.9903, -3.0871,
        -3.0820, -3.0958, -3.0674, -2.9853, -3.0848, -3.0797])
step: tensor(1.00000e-02 *
       [ 4.9489,  4.6028,  4.5984,  ...,  5.0524,  4.5740,  4.5974], device='cuda:0')
ELBO: -130.20372009277344
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 35 ++++++++++
log step: tensor([-3.0056, -3.0804, -3.0818, -3.0615, -3.0782, -2.9771, -3.0849,
        -3.0744, -2.9931, -3.0812, -3.0671, -3.0611, -2.9889, -3.0894,
        -3.0842, -3.0987, -3.0693, -2.9852, -3.0873, -3.0817])
step: tensor(1.00000e-02 *
       [ 4.9507,  4.5941,  4.5877,  ...,  5.0529,  4.5624,  4.

ELBO: -142.076904296875
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 51 ++++++++++
log step: tensor([-3.0257, -3.1155, -3.1124, -3.1020, -3.1158, -2.9863, -3.1254,
        -3.1065, -2.9893, -3.1185, -3.0896, -3.0983, -2.9785, -3.1242,
        -3.1168, -3.1389, -3.0976, -2.9718, -3.1288, -3.1094])
step: tensor(1.00000e-02 *
       [ 4.8524,  4.4358,  4.4495,  ...,  5.1210,  4.3770,  4.4630], device='cuda:0')
ELBO: -138.56178283691406
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 52 ++++++++++
log step: tensor([-3.0276, -3.1172, -3.1138, -3.1038, -3.1180, -2.9861, -3.1273,
        -3.1080, -2.9896, -3.1202, -3.0907, -3.1000, -2.9784, -3.1263,
        -3.1185, -3.1409, -3.0995, -2.9718, -3.1311, -3.1112])
step: tensor(1.00000e-02 *
       [ 4.8432,  4.4279,  4.4432,  ...,  5.1210,  4.3671,  4.45

ELBO: -132.97605895996094
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 68 ++++++++++
log step: tensor([-3.0434, -3.1463, -3.1417, -3.1353, -3.1515, -2.9835, -3.1589,
        -3.1394, -2.9858, -3.1470, -3.1091, -3.1321, -2.9667, -3.1604,
        -3.1386, -3.1717, -3.1209, -2.9639, -3.1645, -3.1394])
step: tensor(1.00000e-02 *
       [ 4.7672,  4.3012,  4.3209,  ...,  5.1615,  4.2236,  4.3308], device='cuda:0')
ELBO: -132.55592346191406
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 1 batch: 69 ++++++++++
log step: tensor([-3.0448, -3.1482, -3.1435, -3.1371, -3.1528, -2.9838, -3.1605,
        -3.1414, -2.9857, -3.1488, -3.1096, -3.1338, -2.9665, -3.1621,
        -3.1396, -3.1741, -3.1219, -2.9640, -3.1661, -3.1413])
step: tensor(1.00000e-02 *
       [ 4.7606,  4.2928,  4.3132,  ...,  5.1613,  4.2168,  4.

...

ELBO: -121.1015396118164
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 3 batch: 907 ++++++++++
log step: tensor([-5.7074, -5.9376, -5.1480, -5.9665, -5.9584, -3.9654, -5.1961,
        -5.0057, -3.0150, -5.1601, -5.2236, -5.9204, -3.6197, -6.1971,
        -5.1105, -5.9274, -6.0838, -3.3419, -5.8388, -6.0503])
step: tensor(1.00000e-02 *
       [ 0.3321,  0.2638,  0.5811,  ...,  3.5369,  0.2912,  0.2357], device='cuda:0')
ELBO: -111.17996978759766
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 3 batch: 908 ++++++++++
log step: tensor([-5.7081, -5.9384, -5.1485, -5.9672, -5.9590, -3.9658, -5.1967,
        -5.0065, -3.0152, -5.1606, -5.2240, -5.9211, -3.6201, -6.1978,
        -5.1110, -5.9282, -6.0844, -3.3421, -5.8394, -6.0509])
step: tensor(1.00000e-02 *
       [ 0.3319,  0.2636,  0.5808,  ...,  3.5361,  0.2911,  0

ELBO: -115.17071533203125
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 3 batch: 923 ++++++++++
log step: tensor([-5.7201, -5.9497, -5.1564, -5.9790, -5.9692, -3.9722, -5.2058,
        -5.0189, -3.0177, -5.1681, -5.2306, -5.9329, -3.6252, -6.2079,
        -5.1188, -5.9400, -6.0945, -3.3456, -5.8502, -6.0598])
step: tensor(1.00000e-02 *
       [ 0.3279,  0.2607,  0.5763,  ...,  3.5239,  0.2879,  0.2335], device='cuda:0')
ELBO: -111.4913558959961
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ Epoch: 3 batch: 924 ++++++++++
log step: tensor([-5.7209, -5.9503, -5.1569, -5.9798, -5.9700, -3.9726, -5.2064,
        -5.0197, -3.0178, -5.1686, -5.2311, -5.9336, -3.6255, -6.2085,
        -5.1192, -5.9408, -6.0951, -3.3458, -5.8510, -6.0604])
step: tensor(1.00000e-02 *
       [ 0.3277,  0.2605,  0.5760,  ...,  3.5230,  0.2877,  0

ELBO: -108.50471496582031
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
++++++++++ test batch: 0 ++++++++++
log step: tensor([-5.7331, -5.9587, -5.1640, -5.9938, -5.9805, -3.9785, -5.2130,
        -5.0322, -3.0202, -5.1756, -5.2408, -5.9436, -3.6299, -6.2177,
        -5.1260, -5.9512, -6.1041, -3.3492, -5.8593, -6.0674])
step: tensor(1.00000e-02 *
       [ 0.3237,  0.2583,  0.5719,  ...,  3.5114,  0.2853,  0.2317], device='cuda:0')
ELBO: -113.24466705322266
++++++++++ test batch: 1 ++++++++++
log step: tensor([-5.7331, -5.9587, -5.1640, -5.9938, -5.9805, -3.9785, -5.2130,
        -5.0322, -3.0202, -5.1756, -5.2408, -5.9436, -3.6299, -6.2177,
        -5.1260, -5.9512, -6.1041, -3.3492, -5.8593, -6.0674])
step: tensor(1.00000e-02 *
       [ 0.3237,  0.2583,  0.5719,  ...,  3.5114,  0.2853,  0.2317], device='cuda:0')
ELBO: -110.35716247558594
++++++++++ test batch: 2 ++++++++++
log step: tensor([-5.7331, -5.9587, -5.16

ELBO: -110.14664459228516
++++++++++ test batch: 21 ++++++++++
log step: tensor([-5.7331, -5.9587, -5.1640, -5.9938, -5.9805, -3.9785, -5.2130,
        -5.0322, -3.0202, -5.1756, -5.2408, -5.9436, -3.6299, -6.2177,
        -5.1260, -5.9512, -6.1041, -3.3492, -5.8593, -6.0674])
step: tensor(1.00000e-02 *
       [ 0.3237,  0.2583,  0.5719,  ...,  3.5114,  0.2853,  0.2317], device='cuda:0')


KeyboardInterrupt: 

In [ ]:
dt = torch.tensor([1., 2.])
tmp = torch.tensor([2., 4.])
x = dt*tmp
x.shape